### Imports

In [ ]:
from agents.fact_checker_agent import FactCheckerAgent
from network.socialnetwork import SocialNetworkEnv
from agents.news_agent import NewsAgent
import numpy as np

### Setup

In [ ]:
network = SocialNetworkEnv(numConsumer=4)

# created a fake and real info agent and add a few consumers to the network
fAgent = NewsAgent("fake-information", state_space_size=2, env=network, trustLevel=0)
rAgent = NewsAgent("real-information", state_space_size=2, env=network, trustLevel=0)
fcAgent = FactCheckerAgent("fact-checker", trustLevel=0.0, env=network)

network.add_news_agents_to_network(agentType=fAgent)
network.add_news_agents_to_network(agentType=rAgent)
network.add_fact_checker_to_network(agentType=fcAgent)

fAgent_action = network.draw_sample_from_action_space()
rAgent_action = network.draw_sample_from_action_space()

print('fagent action', fAgent_action)
print('ragent action', rAgent_action)
# network.render()

# print("---------------- Initial Graph ---------------")
network.print_graph()

### Training

In [ ]:


for i in range(network.network_size):
    print(f"--------- epoch {i} ---------------------")

    obs, reward, penalty = network.step(fAgent_action, fAgent)
    obs2, reward2, penalty2 = network.step(rAgent_action, rAgent)


    network.step_fact_checker(fcAgent)

    network.render()
    # network.print_graph()

    # print("\n\n ------------fake info -------------")
    # print("Observation:", obs)
    # print("Reward:", reward)
    # print("Penalty:", penalty)

    # print("\n\n ------------real info -------------")
    # print("Observation:", obs2)
    # print("Reward:", reward2)
    # print("Info:", penalty2)

    print(f"fake info - reward: {reward}, penalty : {penalty}")
    print(f"real info - reward: {reward2}, penalty : {penalty2}")
    print(f"fact checker - reward: {fcAgent.reward} penalty: {fcAgent.penalty}")
